In [ ]:
%pip install --quiet --upgrade langchain langchain-community langchain-chroma
%pip install -qU langchain-openai

# 使用 LangChain 构建 RAG 代理
LLM 最强大的应用之一是复杂的问答 (Q&A) 聊天机器人。这些应用程序可以回答关于特定来源信息的问题。这些应用程序使用一种称为检索增强生成 (Retrieval Augmented Generation)，或 RAG 的技术。
本教程将展示如何构建一个基于非结构化文本数据源的简单问答应用程序。我们将演示：
一个 RAG 代理，它使用简单的工具执行搜索。这是一个很好的通用实现。
一个两步 RAG 链，每个查询只使用一次 LLM 调用。这对于简单查询来说是一种快速有效的方法。

## 概念
一个典型的RAG应用程序有两个主要组件：

索引：一个从源数据中摄取数据并进行索引的管道。这通常在离线进行。

检索和生成：实际的RAG链，在运行时接收用户查询并从索引中检索相关数据，然后将其传递给模型。

从原始数据到答案的最常见完整序列如下：

## 索引
加载: 首先我们需要加载我们的数据。这是通过 文档加载器 完成的。
分割: 文本分割器 将大型 文档 分割成较小的块。这对于索引数据和将其传递给模型都很有用，因为大型块更难搜索，并且无法适应模型的有限上下文窗口。
存储: 我们需要一个地方来存储和索引我们的分割，以便后续可以进行搜索。这通常使用 向量存储 和 嵌入模型 来完成。

## 检索与生成
检索: 根据用户输入，从存储中使用 检索器 检索相关的分割。
生成: 聊天模型 / 大型语言模型 使用包含问题和检索数据的提示生成答案。

In [ ]:
# 从电脑环境变量中导入
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
print("OPENAI_API_KEY:", OPENAI_API_KEY is not None)
print("LANGCHAIN_API_KEY:", LANGCHAIN_API_KEY is not None)

# 设置环境变量以启用LangChain的跟踪功能
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
# 设置项目名称以便在LangChain跟踪界面中区分不同项目
os.environ["LANGCHAIN_PROJECT"] = "llm-langchain-learn"

# # 兼容 DashScope：将同一个 Key 同步给 DASHSCOPE_API_KEY（若你单独配置了，可删掉此行）
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["DASHSCOPE_API_KEY"] = OPENAI_API_KEY

OPENAI_API_KEY: True
LANGCHAIN_API_KEY: True
sk-85b55aebba56452b8b77e2dbc5006486


In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=OPENAI_API_KEY,
    model="qwen-plus"
)

In [ ]:
# 我们将构建一个应用程序，回答有关网站内容的问题。
import bs4
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=DashScopeEmbeddings()  # 从环境变量 DASHSCOPE_API_KEY 读取 Key
)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

# 直接在本地构造 RAG 提示，避免依赖 langchain.hub
prompt = ChatPromptTemplate.from_template(
    """
You are a helpful assistant. Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know.

Context:
{context}

Question:
{question}

Helpful Answer:
"""
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task Decomposition is the process of breaking down a complex task into smaller, more manageable steps or subtasks. This approach allows for better planning and execution, especially in complicated problem-solving scenarios. In the context of AI and language models, task decomposition can be achieved through methods like Chain of Thought (CoT), where the model is prompted to "think step by step," decomposing the task into simpler reasoning steps. Tree of Thoughts extends this by exploring multiple reasoning paths simultaneously, forming a tree structure of possible thoughts and solutions. Decomposition can be facilitated by prompting the LLM directly, using task-specific instructions, incorporating human input, or even integrating external planners like in the LLM+P framework, which uses PDDL for formal planning. The goal is to enhance performance on complex tasks by structuring them into sequential or hierarchical subgoals.'

# 组件


In [ ]:

import os
from langchain_openai import ChatOpenAI
# 选择模型
model = ChatOpenAI(
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    api_key=OPENAI_API_KEY,
    model="qwen-plus"
)
model.invoke("Hello, world!")

AIMessage(content='Hello! However, there might be a small mix-up. You said "Hello, world!", which is typically something a program or a new AI might say when starting up. But I\'m here and ready to help—how can I assist you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 12, 'total_tokens': 65, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-01f2b63f-3d36-989f-ab93-ac6f8c9a1aea', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019bbbcc-da21-7561-a3fe-bc920ac57f42-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 53, 'total_tokens': 65, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [3]:
# 选择一个向量模型,使用通义千问的 text-embedding-v3 模型（DashScope）
from langchain_community.embeddings import DashScopeEmbeddings

# 使用通义千问的 text-embedding-v3 模型（DashScope）
# 这里和官网的示例稍有不同
embeddings = DashScopeEmbeddings(
    model="text-embedding-v3",
    dashscope_api_key=OPENAI_API_KEY  # 使用你的阿里云 API Key
)

In [4]:
# 选择一个向量存储,langchain_core.vectorstores是
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# 索引
索引通常如下：
加载：首先我们需要加载数据。这是用文档加载器.
分裂：文本分配器把大块拆成更小的块。这对数据索引和数据传递到模型中都很有用，因为大块块更难搜索，且无法放入模型有限的上下文窗口。Documents
存储：我们需要一个地方来存储和索引拆分，以便以后可以搜索。这通常通过VectorStore以及嵌入模特。
## 加载文档
我们需要先加载博客内容。我们可以使用文档加载器对于此，这些对象是从源加载数据并返回列表的对象文件物品。
在这种情况下，我们将使用WebBaseLoader，用于从网页URL加载HTML，并将其解析成文本。我们可以通过向解析器输入参数（参见）来自定义 HTML -> 文本解析urllib BeautifulSoup BeautifulSoupbs_kwargs BeautifulSoup文档).在这种情况下，只有类为“post-content”、“post-title”或“post-header”的HTML标签相关，我们将移除所有其他标签。

In [5]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 仅保留完整的 HTML 中的标题、副标题和内容部分。
bs4_strainer = bs4.SoupStrainer(
    class_=("post-content", "post-title", "post-header")
)
loader = WebBaseLoader(
    web_path="https://lilianweng.github.io/posts/2023-06-23-agent/",
    bs_kwargs=dict(parse_only=bs4_strainer) # 采用过滤器
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43047


In [6]:
print(docs[0].page_content[:500])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In


## 文件拆分
我们的加载文档超过42k字符，太长，无法放入许多模型的上下文窗口。即使是那些能在上下文窗口中容纳完整帖子的模型，模型在非常长的输入中也可能难以找到信息。
为了处理这个问题，我们将分开Document分块用于嵌入和向量存储。这有助于我们在运行时只检索博客文章中最相关的部分。
比如说语义搜索教程我们使用 一个 ，它会递归地使用常用分隔符（如新行）拆分文档，直到每个区块大小合适。这是通用文本用例中推荐的文本拆分器`RecursiveCharacterTextSplitter`。

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 63 sub-documents.


## 存储文件
现在我们需要索引63个文本块，以便在运行时搜索它们。继语义搜索教程，我们的方法是嵌入每个文档的内容会拆分并插入这些嵌入到向量存储.给定输入查询，我们可以使用向量搜索检索相关文档。

In [9]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['c12d8d20-e44f-4c17-b74a-bcfb76f73691', '6e60f09a-581f-43f8-9b72-d9c12ccd4867', '5b098148-76d2-4ce9-84f6-7bdaaa1f8fb5']


# 检索与生成
RAG应用通常如下工作：
检索：给定用户输入，通过以下方式从存储中检索相关拆分.
生成：模型使用包含问题和检索数据的提示生成答案

现在让我们编写实际的应用程序逻辑。我们希望创建一个简单的应用程序，它接受用户问题，搜索与该问题相关的文档，将检索到的文档和初始问题传递给模型，并返回答案。
我们将演示：

一个 RAG 代理，它使用简单的工具执行搜索。这是一个很好的通用实现。
一个两步 RAG 链，每个查询只使用一次 LLM 调用。这对于简单查询来说是一种快速有效的方法。

## RAG代理
RAG 应用程序的一种形式是作为一个简单的 代理，带有一个检索信息的工具。我们可以通过实现一个包装我们向量存储的 工具 来组装一个最小的 RAG 代理。

In [ ]:
from langchain.tools import tool
# 定义一个工具，从向量存储中检索相关内容
@tool(response_format="content_and_artifact")
def retrieve_content(query: str) -> str:
    """Retrieve relevant documents from the vector store and format them."""
    retrieved_docs = vector_store.similarity_search(query, k=3)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent:{doc.page_content}")
        for doc in retrieved_docs
    )

    return serialized, retrieved_docs

In [13]:
# 构建代理
from langchain.agents import create_agent

tools = [retrieve_content]
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(model=model, tools=tools, system_prompt=prompt)

In [14]:
# 我们来测试一下。我们构建了一个通常需要迭代检索步骤才能回答的问题

query = (
    "任务分解的标准方法是什么？\n\n"
"一旦得出答案，请查找该方法的常见扩展形式。"
)

for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

任务分解的标准方法是什么？

一旦得出答案，请查找该方法的常见扩展形式。
================================== Ai Message ==================================
Tool Calls:
  retrieve_content (call_7cf528f4b8eb4131a0e96c)
 Call ID: call_7cf528f4b8eb4131a0e96c
  Args:
    query: 任务分解的标准方法
================================== Ai Message ==================================
Tool Calls:
  retrieve_content (call_7cf528f4b8eb4131a0e96c)
 Call ID: call_7cf528f4b8eb4131a0e96c
  Args:
    query: 任务分解的标准方法
================================= Tool Message =================================
Name: retrieve_content

Source: {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'start_index': 2578}
Content:Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.
A

## RAG链
在上述 代理 RAG 公式中，我们允许 LLM 酌情生成 工具调用 来帮助回答用户查询。这是一个很好的通用解决方案，但也存在一些权衡：
✅ 优点	⚠️ 缺点
仅在需要时搜索——LLM 可以处理问候语、后续问题和简单查询，而不会触发不必要的搜索。	两次推理调用——执行搜索时，需要一次调用来生成查询，另一次调用来生成最终响应。
上下文搜索查询——通过将搜索视为具有 query 输入的工具，LLM 会创建包含对话上下文的自己的查询。	控制减少——LLM 可能会在实际需要时跳过搜索，或者在不必要时发出额外搜索。
允许多次搜索——LLM 可以执行多次搜索以支持单个用户查询。	


另一种常见方法是两步链，我们总是运行一次搜索（可能使用原始用户查询）并将结果作为单个 LLM 查询的上下文。这导致每个查询只进行一次推理调用，以牺牲灵活性为代价降低了延迟。
在这种方法中，我们不再循环调用模型，而是进行一次通过。
我们可以通过从代理中删除工具并将检索步骤合并到自定义提示中来实施此链：

In [16]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest

@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)

    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message


agent = create_agent(model, tools=[], middleware=[prompt_with_context])

In [17]:
query = "What is task decomposition?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is task decomposition?
================================== Ai Message ==================================

Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or steps. This approach helps in understanding and solving complicated problems by focusing on one part at a time. In the context of AI and large language models (LLMs), task decomposition enables better planning and reasoning by allowing the model to "think step by step."

There are several ways to perform task decomposition:

1. **Chain of Thought (CoT)**: The model is prompted to reason through a problem step by step (e.g., "Let's think step by step"), effectively decomposing the task into a linear sequence of simpler reasoning steps.

2. **Tree of Thoughts (ToT)**: An extension of CoT where, at each step, multiple possible thoughts (or reasoning paths) are generated, forming a tree structur

这是一种快速且有效的方法，适用于在受限环境中进行简单查询，而我们通常需要通过语义搜索来获取更多上下文。

以上内容RAG链将检索到的上下文集成到该次运行的单一系统消息中。
比如说能干性RAG在表述阶段，我们有时希望在应用状态中包含原始源文档，以便访问文档元数据。对于两步链的情况，我们可以通过以下方式实现：
向状态添加一个键以存储检索到的文档
通过 添加新节点模型前钩子用来填充那个密钥（同时注入上下文）。

In [ ]:
from typing import Any
from langchain_core.documents import Document
from langchain.agents.middleware import AgentMiddleware, AgentState

# 继承 AgentState（通常包含 messages 等字段），并扩展一个新字段 context，用于存储检索到的文档列表。
# 这样整个 Agent 的状态不仅包含对话消息，还能携带外部检索的上下文。
class State(AgentState):
    context: list[Document]

# 用户提问 → 自动从知识库检索相关文档 → 将文档拼接到问题中 → 送入 LLM 生成答案。
class RetrieveDocumentsMiddleware(AgentMiddleware[State]):
    state_schema = State # 作用是在 Agent 调用模型前/后插入自定义逻辑。

    def before_model(self, state: AgentState) -> dict[str, Any] | None:
        last_message = state["messages"][-1] # 获取用户的最新的提问
        # 进行语义相似性搜索，返回与问题最相关的若干 Document
        retrieved_docs = vector_store.similarity_search(last_message.text)
        # 将所有检索到的文档内容拼接成一个字符串，用空行分隔。
        docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs) 
        # 造增强后的提示（prompt）：原始问题 + 检索到的上下文，并附上指令“Use the following context...”。
        augmented_message_content = (
            f"{last_message.text}\n\n"
            "Use the following context to answer the query:\n"
            f"{docs_content}"
        )
        '''
        返回要更新的状态：
        替换最后一条消息的内容为增强后的版本（注意：这里只保留了最后一条消息，可能丢失历史！）。
        保存 context 到状态中，供后续步骤（如日志、调试、工具调用）使用。
        '''
        return {
            "messages": [last_message.model_copy(update={"content": augmented_message_content})],
            "context": retrieved_docs,
        }


agent = create_agent(
    model,
    tools=[],
    middleware=[RetrieveDocumentsMiddleware()],
)

# 调用agent
query = "What is task decomposition?"
for step in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

Failed to send compressed multipart ingest: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/multipart (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1017)')))"))
Content-Length: 3784
API Key: lsv2_********************************************1atrace=019bbbf6-b5e2-76b3-9b4b-57e0dd71239d,id=019bbbf6-b5e2-76b3-9b4b-57e0dd71239d; trace=019bbbf6-b5e2-76b3-9b4b-57e0dd71239d,id=019bbbf6-b5ef-7361-81c4-3a1b86be3fda; trace=019bbbf6-b5e2-76b3-9b4b-57e0dd71239d,id=019bbbf6-b5ef-7361-81c4-3a1b86be3fda; trace=019bbbf6-b5e2-76b3-9b4b-57e0dd71239d,id=019bbbf6-b7db-7623-89ca-405897770df7; trace=019bbbf6-b5e2-76b3-9b4b-57e0dd71239d,id=019bbbf6-b7dc-77c0-8099-b75bfcb9bd07


================================ Human Message =================================

What is task decomposition?
================================ Human Message =================================

What is task decomposition?

Use the following context to answer the query:
Component One: Planning#
A complicated task usually involves many steps. An agent needs to know what they are and plan ahead.
Task Decomposition#
Chain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.
Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates mul

# 下一步
现在我们已经实现了一个简单的 RAG 应用，通过create_agent我们可以轻松地加入新功能并深入探讨：
流令牌及其他响应式用户体验信息
添加会话记忆支持多回合交互
添加长期记忆支持会话线程间的记忆
添加结构化响应
部署你的应用LangSmith 部署